<div style='background-color: orange'>
<a id='TableOfContents'></a>
    <b><u><i><h1 style='text-align: center'>
        Table of Contents
    </h1></i></u></b>
<li><a href='#imports'>Imports</a></li>
<li><a href='#questions'>Questions To Answer</a></li>
<li><a href='#wrangle'>Wrangle Data</a></li>
<li><a href='#q1'>Question 1</a></li>
<li><a href='#q2'>Question 2</a></li>
<li><a href='#q3'>Question 3</a></li>
<li><a href='#q4'>Question 4</a></li>
<li><a href='#q5'>Question 5</a></li>
<li><a href='#q6'>Question 6</a></li>
<li><a href='#q7'>Question 7</a></li>
<li><a href='#q8'>Question 8</a></li>
<li><a href='#misc'>Miscellaneous</a></li>

<div style='background-color: orange'>
<a id='imports'></a>
    <b><u><i><h1 style='text-align: center'>
        Imports
    </h1></i></u></b>
    <li><a href='#TableOfContents'>Table of Contents</a></li>

In [1]:
# Vectorization & Dataframe
import numpy as np
import pandas as pd

# Visualizations
import matplotlib.pyplot as plt
import seaborn as sns

# stats
from scipy import stats

# Glorious itertools
import itertools


# .py files
import wrangle as w

<div style='background-color: orange'>
<a id='questions'></a>
    <b><u><i><h1 style='text-align: center'>
        Questions To Answer
    </h1></i></u></b>
    <li><a href='#TableOfContents'>Table of Contents</a></li>

1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?
    - <b>ANSWER:</b>

---

2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?
    - <b>ANSWER:</b>

---

3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?
    - <b>ANSWER:</b>

---

4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?
    - <b>ANSWER:</b>

---

5. At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?
    - <b>ANSWER:</b>

---

6. What topics are grads continuing to reference after graduation and into their jobs (for each program)?
    - <b>ANSWER:</b>

---

7. Which lessons are least accessed?
    - <b>ANSWER:</b>

---

8. Anything else I should be aware of?
    - <b>ANSWER:</b>

<div style='background-color: orange'>
<a id='wrangle'></a>
    <b><u><i><h1 style='text-align: center'>
        Wrangle Data
    </h1></i></u></b>
    <li><a href='#TableOfContents'>Table of Contents</a></li>

In [329]:
curr_logs = w.acquire_curriculum_logs()
curr_logs.sample()

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
555322,2020-05-28,09:36:33,mysql/database-design,596,56.0,75.20.162.161,56.0,Apex,#apex,2020-02-24,2020-07-29,2020-02-24 15:39:33,2020-07-29 18:41:13,NaN,2.0


<div style='background-color: orange'>
<a id='q1'></a>
    <b><u><i><h1 style='text-align: center'>
        Question 1
    </h1></i></u></b>
    <li><a href='#TableOfContents'>Table of Contents</a></li>

<b><i>QUESTION 1:</i></b> Which lesson appears to attract the most traffic consistently across cohorts (per program)?

<b><i>ANSWER:</i></b> The introduction lesson is most consistent across all cohorts and has the most frequency from the html-css root.

---

In [330]:
curr_logs['datetime'] = pd.to_datetime(curr_logs.date + ' ' + curr_logs.time)
curr_logs.set_index(curr_logs.datetime, inplace=True)
curr_logs.drop(columns=['date', 'time', 'id', 'slack', 'datetime', 'deleted_at'], inplace=True)
curr_logs.path.fillna('None', inplace=True)
curr_logs.name.fillna('None', inplace=True)
curr_logs.cohort_id.fillna(999, inplace=True)
curr_logs.start_date.fillna('1999-12-31', inplace=True)
curr_logs.end_date.fillna('1999-12-31', inplace=True)
curr_logs.created_at.fillna('1999-12-31', inplace=True)
curr_logs.updated_at.fillna('1999-12-31', inplace=True)
curr_logs.program_id.fillna(999, inplace=True)

In [336]:
curr_logs[curr_logs.program_id == 1.0]

SyntaxError: invalid syntax (25261891.py, line 1)

In [271]:
# Split subjects and lessons from paths
subject = []
lesson = []
for val in curr_logs.path:
    splitted = val.split('/')
    if len(splitted) == 1:
        subject.append(splitted[0])
        lesson.append('None')
    elif len(splitted) == 2:
        if splitted[0] == '':
            subject.append('Home')
            lesson.append('None')
        else:
            subject.append(splitted[0])
            lesson.append(splitted[1])
    elif len(splitted) == 3:
        conditionals = ['content', 'appendix']
        if splitted[0] not in conditionals:
            subject.append(splitted[0])
            lesson.append(splitted[1])
        else:
            subject.append(splitted[1])
            lesson.append(splitted[2])
    elif len(splitted) == 4:
        if splitted[0] == 'appendix':
            subject.append(splitted[2])
            lesson.append(splitted[3])
        elif splitted[0] == 'content':
            subject.append(splitted[1])
            lesson.append(splitted[2])
        else:
            subject.append(splitted[0])
            lesson.append(splitted[1])
    elif len(splitted) == 5:
        subject.append(splitted[1])
        lesson.append(splitted[2])
    elif len(splitted) == 6:
        if splitted[1] == 'appendix':
            subject.append(splitted[2])
            lesson.append(splitted[3])
        else:
            subject.append(splitted[1])
            lesson.append(splitted[2])
    elif len(splitted) == 7:
        subject.append(splitted[2])
        lesson.append(splitted[3])
    elif len(splitted) == 8:
        subject.append('Anomaly')
        lesson.append('Anomaly')
curr_logs['subject'] = subject
curr_logs['lesson'] = lesson

In [24]:
# Fill any 'na' with 'None'
curr_logs.path.fillna('None', inplace=True)

In [145]:
# Most viewed 'end_path' per cohort
pd.DataFrame(curr_logs[curr_logs.end_path != ''].groupby(by='name').end_path.value_counts(normalize=True).groupby(level=0).nlargest(1))

,,,end_path
name,name,end_path,
Andromeda,Andromeda,introduction,0.032913
Apex,Apex,search_index.json,0.046459
Apollo,Apollo,html-css,0.500000
Arches,Arches,javascript-i,0.035576
Badlands,Badlands,command-line,0.078947
Bash,Bash,introduction,0.039372
Bayes,Bayes,1-overview,0.074315
Betelgeuse,Betelgeuse,introduction,0.030457
Ceres,Ceres,introduction,0.037234


In [150]:
# Count of 'root_paths' for 'introduction'
curr_logs[curr_logs.end_path == 'introduction'].root_path.value_counts()

html-css        8165
mysql           5082
jquery          4408
javascript-i    3434
java-iii         502
introduction      63
css-i              2
mvc                2
Name: root_path, dtype: int64

In [146]:
# Count of 'end_path' frequency
curr_logs.end_path.value_counts(normalize=True)

                              0.055889
introduction                  0.024058
search_index.json             0.023805
javascript-i                  0.021087
toc                           0.020325
                                ...   
threads                       0.000001
coding_best_practices         0.000001
interview-questions           0.000001
interview                     0.000001
post-interview-review-form    0.000001
Name: end_path, Length: 1509, dtype: float64

In [156]:
curr_logs[curr_logs.path != '/'].groupby('name').path.value_counts().groupby(level=0).nlargest(1)

name        name        path                                    
Andromeda   Andromeda   toc                                          638
Apex        Apex        search/search_index.json                    1497
Apollo      Apollo      content/html-css                               2
Arches      Arches      javascript-i                                 294
Badlands    Badlands    content/php_ii/command-line                    6
Bash        Bash        search/search_index.json                     660
Bayes       Bayes       1-fundamentals/modern-data-scientist.jpg     650
Betelgeuse  Betelgeuse  search/search_index.json                     761
Ceres       Ceres       search/search_index.json                    1380
Curie       Curie       6-regression/1-overview                      595
Darden      Darden      classification/overview                     1109
Deimos      Deimos      javascript-i                                 896
Denali      Denali      mkdocs/search_index.json           

<div style='background-color: orange'>
<a id='q2'></a>
    <b><u><i><h1 style='text-align: center'>
        Question 2
    </h1></i></u></b>
    <li><a href='#TableOfContents'>Table of Contents</a></li>

<b><i>QUESTION 2:</i></b>  Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?

<b><i>ANSWER:</i></b> 

---

In [175]:
curr_logs[curr_logs.end_path != ''].end_path.value_counts(normalize=True).head(10)

introduction         0.025483
search_index.json    0.025214
javascript-i         0.022335
toc                  0.021528
html-css             0.016792
java-iii             0.016159
java-ii              0.014925
spring               0.014746
functions            0.014331
jquery               0.013649
Name: end_path, dtype: float64

<div style='background-color: orange'>
<a id='q3'></a>
    <b><u><i><h1 style='text-align: center'>
        Question 3
    </h1></i></u></b>
    <li><a href='#TableOfContents'>Table of Contents</a></li>

<b><i>QUESTION 3:</i></b> Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?

<b><i>ANSWER:</i></b> 

---

<div style='background-color: orange'>
<a id='q4'></a>
    <b><u><i><h1 style='text-align: center'>
        Question 4
    </h1></i></u></b>
    <li><a href='#TableOfContents'>Table of Contents</a></li>

<b><i>QUESTION 4:</i></b> Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?

<b><i>ANSWER:</i></b> 

---

<div style='background-color: orange'>
<a id='q5'></a>
    <b><u><i><h1 style='text-align: center'>
        Question 5
    </h1></i></u></b>
    <li><a href='#TableOfContents'>Table of Contents</a></li>

<b><i>QUESTION 5:</i></b> At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?

<b><i>ANSWER:</i></b> 

---

<div style='background-color: orange'>
<a id='q6'></a>
    <b><u><i><h1 style='text-align: center'>
        Question 6
    </h1></i></u></b>
    <li><a href='#TableOfContents'>Table of Contents</a></li>

<b><i>QUESTION 6:</i></b> What topics are grads continuing to reference after graduation and into their jobs (for each program)?

<b><i>ANSWER:</i></b> 

---

<div style='background-color: orange'>
<a id='q7'></a>
    <b><u><i><h1 style='text-align: center'>
        Question 7
    </h1></i></u></b>
    <li><a href='#TableOfContents'>Table of Contents</a></li>

<b><i>QUESTION 7:</i></b> Which lessons are least accessed?

<b><i>ANSWER:</i></b> 

---

<div style='background-color: orange'>
<a id='q8'></a>
    <b><u><i><h1 style='text-align: center'>
        Question 8
    </h1></i></u></b>
    <li><a href='#TableOfContents'>Table of Contents</a></li>

<b><i>QUESTION 8:</i></b> Anything else I should be aware of?

<b><i>ANSWER:</i></b> 

---

<div style='background-color: orange'>
<a id='misc'></a>
    <b><u><i><h1 style='text-align: center'>
        Miscellaneous
    </h1></i></u></b>
    <li><a href='#TableOfContents'>Table of Contents</a></li>

In [152]:
longest = []
for val in curr_logs.path:
    stuff = val.split('/')
    if len(stuff) > len(longest):
        longest = stuff
print(longest)

['cohorts', '%7Bid%7D', 'grades.csvhttp:', '', 'tools.codeup.com', 'cohorts', '26', 'grades.csv']
